<a href="https://colab.research.google.com/github/elliot-brooks/nlu-coursework/blob/main/src/AV_SVM_TRAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -U transformers
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 28.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel
import torch
import re

# Load training data

In [ ]:
training_corpus = pd.read_csv("train.csv", encoding='utf-8')

# Pre-process training data

In [ ]:
def preprocess(string):
  output = str(string).lower()
  separated_string = re.sub(r'([^\w\s])', r' \1 ', str(string))
  return output

# Prepare data for Distilled Bert
def prepare_data(data) :
  data["text_1"] = data["text_1"].apply(lambda x: preprocess(x))
  data["text_2"] = data["text_2"].apply(lambda x: preprocess(x))
  concat_pairs = []
  for index, row in data.iterrows():
      concatenated_pair = row["text_1"] + " [SEP] " + row["text_2"]
      concat_pairs.append(concatenated_pair)
  return concat_pairs

concat_data = prepare_data(training_corpus)

Load BERT Models

In [ ]:
tokeniser = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Create BERT Embeddings

In [ ]:
SEQ_LENGTH = 128
BATCH_SIZE = 32

def create_bert_embeddings_batch(texts, tokeniser, model, batch_size, seq_length) :
  embeddings = []
  for i in range(0, len(texts), batch_size) :
    batch = texts[i:i + batch_size]
    inputs = tokeniser.batch_encode_plus(batch, padding='max_length', truncation=True, return_tensors='tf', max_length=seq_length, add_special_tokens=True)

    # Create embeddings
    outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])

    last_hidden_state_CLS = outputs.last_hidden_state[:, 0, :]

    embeddings.append(last_hidden_state_CLS)
  return embeddings

bert_embeddings = create_bert_embeddings_batch(concat_data, tokeniser, bert_model, BATCH_SIZE, SEQ_LENGTH)
train_labels = np.array(training_corpus['label'])

# Define Language Model

In [ ]:
BERT_ENCODING_DIM = 768

LEARNING_RATE = 5e-5

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(BERT_ENCODING_DIM,)),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               393728    
                                                                 
 dropout_19 (Dropout)        (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_20 (Dropout)        (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_21 (Dropout)        (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8

# Train Model

In [ ]:
num_samples = train_labels.shape[0]



# Concatenate BERT batches
print(len(bert_embeddings))
print(len(bert_embeddings[0]))
print(len(bert_embeddings[0][0]))

train_inputs = np.concatenate(bert_embeddings, axis=0)
print(train_inputs.shape)

model.fit(train_inputs, train_labels, epochs=50, batch_size=64, validation_split=0.1)

(30000, 768)
Epoch 1/50
422/422 [==============================] - 19s 5ms/step - loss: 10.1568 - accuracy: 0.4978 - val_loss: 7.8346 - val_accuracy: 0.4920
Epoch 2/50
422/422 [==============================] - 2s 4ms/step - loss: 6.1983 - accuracy: 0.4952 - val_loss: 4.8083 - val_accuracy: 0.4930
Epoch 3/50
422/422 [==============================] - 2s 4ms/step - loss: 3.8525 - accuracy: 0.4991 - val_loss: 3.0516 - val_accuracy: 0.4930
Epoch 4/50
422/422 [==============================] - 2s 4ms/step - loss: 2.5082 - accuracy: 0.4991 - val_loss: 2.0545 - val_accuracy: 0.4947
Epoch 5/50
422/422 [==============================] - 2s 4ms/step - loss: 1.7462 - accuracy: 0.4994 - val_loss: 1.4879 - val_accuracy: 0.4973
Epoch 6/50
422/422 [==============================] - 2s 4ms/step - loss: 1.3095 - accuracy: 0.5031 - val_loss: 1.1592 - val_accuracy: 0.5053
Epoch 7/50
422/422 [==============================] - 2s 4ms/step - loss: 1.0536 - accuracy: 0.4979 - val_loss: 0.9639 - val_accuracy

# Save Model

In [ ]:
model.save("AV_SVM_MODEL")

In [ ]:
!zip -r /content/SVM_MODEL.zip /content/AV_SVM_MODEL

  adding: content/AV_SVM_MODEL/ (stored 0%)
  adding: content/AV_SVM_MODEL/fingerprint.pb (stored 0%)
  adding: content/AV_SVM_MODEL/keras_metadata.pb (deflated 90%)
  adding: content/AV_SVM_MODEL/saved_model.pb (deflated 88%)
  adding: content/AV_SVM_MODEL/variables/ (stored 0%)
  adding: content/AV_SVM_MODEL/variables/variables.data-00000-of-00001 (deflated 27%)
  adding: content/AV_SVM_MODEL/variables/variables.index (deflated 64%)
  adding: content/AV_SVM_MODEL/assets/ (stored 0%)
